# RCNN - model for Classification

### Resources
<a href="https://pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/">IOU</a>

<a href="https://towardsdatascience.com/step-by-step-r-cnn-implementation-from-scratch-in-python-e97101ccde55">Overall Guide</a>

### IOU
* this is just an evaluation metric
* this requires two things:
    * ground truth boxes - the hand labeld bounding boxes from the testing set that specify where in the image our object is.
    * the predicted vounding boxes from our model
    
    IOU= (area of overlap) / (area of union)

In [1]:
# all the required lib
import os, cv2, keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

2022-04-26 18:25:41.544357: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dhruv/.conda/envs/ml/lib/python3.10/site-packages/cv2/../../lib64:
2022-04-26 18:25:41.544448: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
ss= cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [3]:
def get_iou(bb1, bb2):
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [4]:
cd

/home/dhruv


In [5]:
cd drone-defense-wall/data

/home/dhruv/drone-defense-wall/data


In [6]:
cd airplanes

/home/dhruv/drone-defense-wall/data/airplanes


In [7]:
pwd

'/home/dhruv/drone-defense-wall/data/airplanes'

In [8]:
ls

airplane_416.jpg  airplane_492.jpg  airplane_568.jpg  airplane_644.jpg
airplane_417.jpg  airplane_493.jpg  airplane_569.jpg  airplane_645.jpg
airplane_418.jpg  airplane_494.jpg  airplane_570.jpg  airplane_646.jpg
airplane_419.jpg  airplane_495.jpg  airplane_571.jpg  airplane_647.jpg
airplane_420.jpg  airplane_496.jpg  airplane_572.jpg  airplane_648.jpg
airplane_421.jpg  airplane_497.jpg  airplane_573.jpg  airplane_649.jpg
airplane_422.jpg  airplane_498.jpg  airplane_574.jpg  airplane_650.jpg
airplane_423.jpg  airplane_499.jpg  airplane_575.jpg  airplane_651.jpg
airplane_424.jpg  airplane_500.jpg  airplane_576.jpg  airplane_652.jpg
airplane_425.jpg  airplane_501.jpg  airplane_577.jpg  airplane_653.jpg
airplane_426.jpg  airplane_502.jpg  airplane_578.jpg  airplane_654.jpg
airplane_427.jpg  airplane_503.jpg  airplane_579.jpg  airplane_655.jpg
airplane_428.jpg  airplane_504.jpg  airplane_580.jpg  airplane_656.jpg
airplane_429.jpg  airplane_505.jpg  airplane_581.jpg  airplane_657.jpg
airpla

In [10]:
path = "Images"
annot = "Airplanes_Annotations"

In [11]:
train_images=[]
train_labels=[]
for e,i in enumerate(os.listdir(annot)):
    try:
        if i.startswith("airplane"):
            filename = i.split(".")[0]+".jpg"
            print(e,filename)
            image = cv2.imread(os.path.join(path,filename))
            df = pd.read_csv(os.path.join(annot,i))
            gtvalues=[]
            for row in df.iterrows():
                x1 = int(row[1][0].split(" ")[0])
                y1 = int(row[1][0].split(" ")[1])
                x2 = int(row[1][0].split(" ")[2])
                y2 = int(row[1][0].split(" ")[3])
                gtvalues.append({"x1":x1,"x2":x2,"y1":y1,"y2":y2})
            ss.setBaseImage(image)
            ss.switchToSelectiveSearchFast()
            ssresults = ss.process()
            imout = image.copy()
            counter = 0
            falsecounter = 0
            flag = 0
            fflag = 0
            bflag = 0
            for e,result in enumerate(ssresults):
                if e < 2000 and flag == 0:
                    for gtval in gtvalues:
                        x,y,w,h = result
                        iou = get_iou(gtval,{"x1":x,"x2":x+w,"y1":y,"y2":y+h})
                        if counter < 30:
                            if iou > 0.70:
                                timage = imout[y:y+h,x:x+w]
                                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                                train_images.append(resized)
                                train_labels.append(1)
                                counter += 1
                        else :
                            fflag =1
                        if falsecounter <30:
                            if iou < 0.3:
                                timage = imout[y:y+h,x:x+w]
                                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                                train_images.append(resized)
                                train_labels.append(0)
                                falsecounter += 1
                        else :
                            bflag = 1
                    if fflag == 1 and bflag == 1:
                        print("inside")
                        flag = 1
    except Exception as e:
        print(e)
        print("error in "+filename)
        continue

0 airplane_435.jpg


[ WARN:0@99.485] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('Images/airplane_435.jpg'): can't open/read file: check file path/integrity


OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

error in airplane_435.jpg
1 airplane_104.jpg
2 airplane_040.jpg
3 airplane_021.jpg
4 airplane_461.jpg
OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

error in airplane_461.jpg
5 airplane_271.jpg


[ WARN:0@103.622] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('Images/airplane_461.jpg'): can't open/read file: check file path/integrity


6 airplane_404.jpg
7 airplane_599.jpg
OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

error in airplane_599.jpg
8 airplane_262.jpg


...

726 airplane_610.jpg
OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

error in airplane_610.jpg
727 airplane_455.jpg
OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

error in airplane_455.jpg
728 airplane_186.jpg


[ WARN:0@625.722] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('Images/airplane_610.jpg'): can't open/read file: check file path/integrity
[ WARN:0@625.725] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('Images/airplane_455.jpg'): can't open/read file: check file path/integrity


729 airplane_341.jpg
730 airplane_187.jpg
731 airplane_530.jpg


[ WARN:0@628.003] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('Images/airplane_530.jpg'): can't open/read file: check file path/integrity


OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

error in airplane_530.jpg
732 airplane_259.jpg


In [12]:
X_new = np.array(train_images)
y_new = np.array(train_labels)

In [ ]:
X_new.shape

In [ ]:
from keras.layers import Dense
from keras import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.applications.vgg16 import VGG16


In [ ]:
vggmodel = VGG16(weights='imagenet', include_top=True)
vggmodel.summary()

In [ ]:
for layers in (vggmodel.layers)[:15]:
    print(layers)
    layers.trainable = False

In [ ]:
X= vggmodel.layers[-2].output

In [ ]:
predictions = Dense(2, activation="softmax")(X)

In [ ]:
model_final = Model(input = vggmodel.input, output = predictions)

In [ ]:
from keras.optimizers import Adam
opt = Adam(lr=0.0001)

In [ ]:
model_final.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])

In [ ]:
model_final.summary()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [ ]:
class MyLabelBinarizer(LabelBinarizer):
    def transform(self, y):
        Y = super().transform(y)
        if self.y_type_ == 'binary':
            return np.hstack((Y, 1-Y))
        else:
            return Y
    def inverse_transform(self, Y, threshold=None):
        if self.y_type_ == 'binary':
            return super().inverse_transform(Y[:, 0], threshold)
        else:
            return super().inverse_transform(Y, threshold)


In [ ]:
lenc = MyLabelBinarizer()
Y =  lenc.fit_transform(y_new)

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(X_new,Y,test_size=0.10)

In [ ]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
trdata = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
traindata = trdata.flow(x=X_train, y=y_train)
tsdata = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
testdata = tsdata.flow(x=X_test, y=y_test)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
checkpoint = ModelCheckpoint("ieeercnn_vgg16_1.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')

In [ ]:
hist = model_final.fit_generator(generator= traindata, steps_per_epoch= 10, epochs= 1000, validation_data= testdata, validation_steps=2, callbacks=[checkpoint,early])

In [ ]:
import matplotlib.pyplot as plt
# plt.plot(hist.history["acc"])
# plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Loss","Validation Loss"])
plt.show()
plt.savefig('chart loss.png')

In [ ]:
im = X_test[1600]
plt.imshow(im)
img = np.expand_dims(im, axis=0)
out= model_final.predict(img)
if out[0][0] > out[0][1]:
    print("plane")
else:
    print("not plane")

In [ ]:
z=0
for e,i in enumerate(os.listdir(path)):
    if i.startswith("4"):
        z += 1
        img = cv2.imread(os.path.join(path,i))
        ss.setBaseImage(img)
        ss.switchToSelectiveSearchFast()
        ssresults = ss.process()
        imout = img.copy()
        for e,result in enumerate(ssresults):
            if e < 2000:
                x,y,w,h = result
                timage = imout[y:y+h,x:x+w]
                resized = cv2.resize(timage, (224,224), interpolation = cv2.INTER_AREA)
                img = np.expand_dims(resized, axis=0)
                out= model_final.predict(img)
                if out[0][0] > 0.65:
                    cv2.rectangle(imout, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
        plt.figure()
        plt.imshow(imout)